In [6]:
!pip install ultralytics --quiet
!pip install -U ipywidgets --quiet
!pip uninstall wandb -y

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
Found existing installation: wandb 0.18.3
Uninstalling wandb-0.18.3:
  Successfully uninstalled wandb-0.18.3


In [7]:
import os
import shutil
from distutils.dir_util import copy_tree

from ultralytics.data.utils import autosplit
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# HYPERPARAMETERS
TRAIN_FRACTION = 0.8
VAL_FRACTION = 0.1
TEST_FRACTION = 0.1
EPOCHS = 200

assert ((TRAIN_FRACTION + VAL_FRACTION + TEST_FRACTION) == 1.0), "Sum of split fraction != 1"

In [5]:
# Copy train images from kaggle input directory to kaggle output directory
dataset_source_dir = '/kaggle/input/fixed-wing-uavs/dataset-07-10-2024/'
dataset_target_dir = '/kaggle/working/dataset/'

data_yaml_source_path = '/kaggle/working/dataset/data_kaggle.yaml'
data_yaml_target_path = '/kaggle/working/data_kaggle.yaml'

os.makedirs(dataset_target_dir, exist_ok=True)
copy_tree(dataset_source_dir, dataset_target_dir)
shutil.move(data_yaml_source_path, data_yaml_target_path)

'/kaggle/working/data_kaggle.yaml'

In [6]:
# create data split
autosplit(path="/kaggle/working/dataset", weights = (TRAIN_FRACTION, VAL_FRACTION, TEST_FRACTION))

Autosplitting images from /kaggle/working/dataset


100%|██████████| 891/891 [00:00<00:00, 19106.25it/s]


In [7]:
# use pre-trained yolov11 model
model = YOLO("yolo11n.pt")
results = model.train(data="/kaggle/working/data_kaggle.yaml",
                      epochs=EPOCHS,
                      device=[0, 1],
                      name='different_split')

Ultralytics 8.3.18 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data_kaggle.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=different_split, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

100%|██████████| 755k/755k [00:00<00:00, 14.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/dataset... 590 images, 132 backgrounds, 1 corrupt: 100%|██████████| 723/723 [00:00<00:00, 859.22it/s]


train: WARNING ⚠️ /kaggle/working/dataset/517.jpg: ignoring corrupt image/label: cannot identify image file '/kaggle/working/dataset/517.jpg'
train: New cache created: /kaggle/working/dataset.cache


...

                   all         78         82      0.928      0.793       0.82      0.661

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    200/200      1.26G     0.2642     0.2259     0.8565          1        640: 100%|██████████| 46/46 [00:09<00:00,  4.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  9.30it/s]


                   all         78         82      0.929      0.803      0.823      0.665

200 epochs completed in 0.633 hours.
Optimizer stripped from runs/detect/different_split/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/different_split/weights/best.pt, 5.5MB

Validating runs/detect/different_split/weights/best.pt...
Ultralytics 8.3.18 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.08it/s]


                   all         78         82      0.901      0.793      0.816      0.678
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/different_split


In [2]:
import random

def generate_random_image_paths(num_paths):
    with open("/kaggle/working/autosplit_test.txt", "r") as f:
        data = f.read().replace('./', '/kaggle/working/').split('\n')
        data = data[:-1]
    
    generated_paths = []
    image_nums = random.sample(range(0, len(data)), num_paths)

    for i in image_nums:
        generated_paths.append(data[i])
        
    return generated_paths

In [8]:
paths = generate_random_image_paths(60)
print(paths)

['/kaggle/working/dataset/522.jpg', '/kaggle/working/dataset/149.jpg', '/kaggle/working/dataset/577.jpg', '/kaggle/working/dataset/857.jpg', '/kaggle/working/dataset/94.jpg', '/kaggle/working/dataset/113.jpg', '/kaggle/working/dataset/441.jpg', '/kaggle/working/dataset/228.jpg', '/kaggle/working/dataset/17.jpg', '/kaggle/working/dataset/363.jpg', '/kaggle/working/dataset/867.jpg', '/kaggle/working/dataset/124.jpg', '/kaggle/working/dataset/639.jpg', '/kaggle/working/dataset/443.jpg', '/kaggle/working/dataset/740.jpg', '/kaggle/working/dataset/65.jpg', '/kaggle/working/dataset/297.jpg', '/kaggle/working/dataset/619.jpg', '/kaggle/working/dataset/741.jpg', '/kaggle/working/dataset/568.jpg', '/kaggle/working/dataset/232.jpg', '/kaggle/working/dataset/360.jpg', '/kaggle/working/dataset/608.jpg', '/kaggle/working/dataset/143.jpg', '/kaggle/working/dataset/521.jpg', '/kaggle/working/dataset/784.jpg', '/kaggle/working/dataset/535.jpg', '/kaggle/working/dataset/80.jpg', '/kaggle/working/datase

In [ ]:
model = YOLO('/kaggle/working/runs/detect/different_split/weights/best.pt')
results = model(paths, batch=16, device=0,save=True, project='fixed-wing-uavs', name='results')
for result in results:
    result.show()  # display to screen
    